<a href="https://colab.research.google.com/github/Malanidhruv/assets/blob/main/FINNIFTY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import datetime
from datetime import datetime, timedelta, time
import holidays
from pya3 import *
import mysql.connector
import re

def lambda_handler(event, context):

    # Database connection details
    DB_NAME = 'bitnami_wordpress'
    DB_USER = 'bn_wordpress'
    DB_PASSWORD = '794218776da450c31493d660c391d93d9d5b395d60d48f3aa6abbd81ec29d9d5'
    DB_HOST = '3.109.88.68'
    DB_PORT = 3306

    try:
        # Connect to the database
        conn = mysql.connector.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME
        )

        # Check if the connection is successful
        if conn.is_connected():
            print("Connected to the database!")
            # Perform database operations
            cursor = conn.cursor()
            cursor.execute("SELECT * FROM wp_wpforms_db")
            rows = cursor.fetchall()

            # Filter rows where the second column value is either 68 or 145
            filtered_rows = [row for row in rows if row[1] in [82, 145]]
            print("Filtered Rows:", filtered_rows)

            user_api_dict = {}
            for row in filtered_rows:
                print("Row[2] Value:", row[2])  # Debugging statement
                match = re.search(r'"User ID";s:(\d+):"(\d+)".*?"API Key";s:(\d+):"([^"]+)".*?"Lot Size";s:(\d+):"(\d+)"', row[2])
                if match:
                    user_id = match.group(2)
                    api_key = match.group(4)
                    lot_size = int(match.group(6))  # Correct key is 'lot_size'
                    user_api_dict[user_id] = {'api_key': api_key, 'lot_size': lot_size}

            print("User API Dictionary:", user_api_dict)

            # Close the cursor and connection when done
            cursor.close()
            conn.close()

    except mysql.connector.Error as err:
        print("Failed to connect to the database:", err)
    except KeyError as e:
        print(f"KeyError: {e}. Please check the keys in the user_api_dict.")

    def order_checker():
        user_id1 = "1141826"
        api_key1 = user_api_dict.get(user_id1, {}).get('api_key')
        if not api_key1:
            print("API Key not found for user_id1")
            return

        # Create an Aliceblue instance and get the session ID
        global alice
        alice = Aliceblue(user_id=user_id1, api_key=api_key1)
        alice.get_session_id()

        from nsetools import Nse
        import holidays
        import datetime

        # Create an instance of Nse
        nse = Nse()

        # Get the list of market holidays for the year
        market_holidays = holidays.India(years=datetime.datetime.now().year)

        # Add additional holidays if needed
        additional_holidays = [
            datetime.date(2024, 4, 18),
            datetime.date(2024, 4, 17),
            datetime.date(2024, 5, 20)
        ]
        for holiday in additional_holidays:
            market_holidays[holiday] = "Holiday"

        # Get the current date
        current_date = datetime.datetime.now()

        # Find the previous trading day
        previous_trading_day = current_date - timedelta(days=1)
        while previous_trading_day.weekday() >= 5 or previous_trading_day.date() in market_holidays:
            previous_trading_day -= timedelta(days=1)

        yesterday_date = previous_trading_day.date()

        # Define the instrument symbol directly
        instrument = alice.get_instrument_by_symbol('NFO', "FINNIFTY30JUL24F")
        print(instrument)

        from_datetime = datetime.datetime.combine(yesterday_date, datetime.time.min)  # From yesterday
        to_datetime = datetime.datetime.combine(yesterday_date, datetime.time.max)  # To yesterday
        interval = "D"  # ["1", "D"]

        historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
        print(historical_data)

        last_close_price = historical_data['close'].iloc[-1]
        print("Closing Price of the Last Minute:", last_close_price)

        # Store the closing price for later use
        closeprice = last_close_price

        # Now you can use 'stored_close_price' in your subsequent code
        print("Closing Price:", closeprice)

        # Function to check if a given date is a holiday
        def is_holiday(date):
            india_holidays = holidays.CountryHoliday('IND', prov='MH')
            return date in india_holidays

        def next_tuesday(current_date):
            # Calculate days until next Tuesday (weekday 1)
            days_until_next_tuesday = (1 - current_date.weekday() + 7) % 7
            if days_until_next_tuesday == 0:
                days_until_next_tuesday = 7  # If today is Tuesday, go to the next Tuesday
            next_tuesday = current_date + timedelta(days=days_until_next_tuesday)

            # Check for holiday adjustment
            if is_holiday(next_tuesday):
                next_tuesday -= timedelta(days=1)
            return next_tuesday

        # Assuming you have the closing price stored in 'last_close_price'
        closing_price = closeprice  # Replace this with the actual closing price from your data

        # Calculate the nearest strike prices
        ce_strike_price = int(closing_price / 50) * 50
        pe_strike_price = ce_strike_price + 50

        # Get today's date
        today = datetime.date.today()

        # Check if it's the last week of the month
        is_last_week_of_month = (today + timedelta(days=7)).month != today.month

        # Check if it's the last weekly expiry
        is_last_weekly_expiry = today.weekday() == 3 and is_last_week_of_month

        # Get the next Thursday for expiry date based on the current date and the condition
        if is_last_weekly_expiry:
            expiry_date = today
        else:
            expiry_date = next_tuesday(today)

        # Construct option symbols with expiry date and strike prices
        global symbol_ce
        global symbol_pe

        symbol_ce = f'FINNIFTY{expiry_date.strftime("%d%b%y")}C{ce_strike_price}'
        symbol_pe = f'FINNIFTY{expiry_date.strftime("%d%b%y")}P{pe_strike_price}'

        print("CE Option Symbol:", symbol_ce)
        print("PE Option Symbol:", symbol_pe)

        # Define the instrument symbol_ce
        callopf = alice.get_instrument_by_symbol('NFO', symbol_ce)

        # Get historical data
        historical_data = alice.get_historical(callopf, from_datetime, to_datetime, interval)

        # Assuming 'data' is a DataFrame
        high_values = historical_data['high'].tolist()

        global highest_high_ce
        global openprice_ce

        highest_high_ce = historical_data['high'].max()

        highest_high_ce = round(highest_high_ce)
        highest_high_ce = float(highest_high_ce)
        print("Rounded High Value:", highest_high_ce)

        # Define the start and end times for 9:15 AM to 9:16 AM
        current_date = datetime.date.today()
        from_datetime1 = datetime.datetime.combine(current_date, time(9, 15))
        to_datetime1 = datetime.datetime.combine(current_date, time(9, 16))
        interval = "1"  # ["1", "D"]

        # Fetch historical data for the specified interval
        historical_data = alice.get_historical(callopf, from_datetime1, to_datetime1, interval)

        # Extract the opening price of the first minute
        opening_price = historical_data['open'].iloc[0]
        print("Opening Price of the First Minute:", opening_price)

        # Store the opening price for later use
        openprice_ce = opening_price

        # Define the instrument symbol_pe
        putopf = alice.get_instrument_by_symbol('NFO', symbol_pe)

        # Get historical data
        historical_data = alice.get_historical(putopf, from_datetime, to_datetime, interval)

        # Assuming 'data' is a DataFrame
        high_values = historical_data['high'].tolist()

        global highest_high_pe
        global openprice_pe
        #################################################################
        highest_high_pe = historical_data['high'].max()

        highest_high_pe = round(highest_high_pe)
        highest_high_pe = float(highest_high_pe)
        print("Rounded High Value:", highest_high_pe)

        # Define the start and end times for 9:15 AM to 9:16 AM
        current_date = datetime.date.today()
        from_datetime1 = datetime.datetime.combine(current_date, time(9, 15))
        to_datetime1 = datetime.datetime.combine(current_date, time(9, 16))
        interval = "1"  # ["1", "D"]

        # Fetch historical data for the specified interval
        historical_data = alice.get_historical(putopf, from_datetime1, to_datetime1, interval)


        # Extract the opening price of the first minute
        opening_price = historical_data['open'].iloc[0]
        print("Opening Price of the First Minute:", opening_price)

        # Store the opening price for later use
        openprice_pe = opening_price

    def order_placer(user, api, quantity):
        alice = Aliceblue(user_id=user, api_key=api)
        alice.get_session_id()
        global symbol_ce
        global highest_high_ce
        global openprice_ce

        # Placing order for ce
        CE_high = highest_high_ce
        price_value = CE_high

        if openprice_ce <= price_value:
            order_response = alice.place_order(transaction_type=TransactionType.Buy,
                                               instrument=alice.get_instrument_by_symbol('NFO', symbol_ce),
                                               quantity=quantity*25,
                                               order_type=OrderType.StopLossLimit,
                                               product_type=ProductType.BracketOrder,
                                               price=price_value,
                                               trigger_price=price_value,
                                               stop_loss=40.0,
                                               square_off=40.0,
                                               trailing_sl=20.0,
                                               is_amo=False,
                                               order_tag='order1')
            print("Order placed successfully, no gap up detected.", order_response)
        else:
            print("Opening price is greater than the highest high price. Gap up detected. Order not placed.")

        # Placing order for pe
        global symbol_pe
        global highest_high_pe
        global openprice_pe

        PE_high = highest_high_pe
        price_value = PE_high
        PE_high = "highest_high_pe"

        # Retrieve the variable value using globals()
        price_value = globals()[PE_high]
        if openprice_pe > highest_high_pe:
            print("No order placed because there is a gap up.")

        else:
            final_order = alice.place_order(transaction_type=TransactionType.Buy,
                                            instrument=alice.get_instrument_by_symbol('NFO', symbol_pe),
                                            quantity=quantity*25,
                                            order_type=OrderType.StopLossLimit,
                                            product_type=ProductType.BracketOrder,
                                            price=price_value,
                                            trigger_price=price_value,
                                            stop_loss=40.0,
                                            square_off=40.0,
                                            trailing_sl=20.0,
                                            is_amo=False,
                                            order_tag='order1')
            print("Order placed successfully for PE:", final_order)

    order_checker()

    # Iterate through user_api_dict and call order_placer function
    for user, api in user_api_dict.items():
        print("User ID:", user)
        print("API Key:", api['api_key'])
        order_placer(user, api['api_key'], api['lot_size'])

    return {
        'statusCode': 200,
        'body': json.dumps({'message': 'Orders placed successfully'})
    }
